In [12]:
from policyengine_us import Simulation
from policyengine_core.reforms import Reform
from policyengine_core.charts import *
import plotly.graph_objects as go

In [13]:
reform = Reform.from_dict({
  "gov.contrib.congress.golden.fisc_act.in_effect": {
    "2026-01-01.2100-12-31": 0.01
  }
}, country_id="us")



In [14]:
# Constants
MAX_INCOME = 180000
YEAR = "2026"  

In [15]:
def create_situation(child_age):
    # Create people dictionary with adult and one child of specified age
    people = {
        "adult": {"age": {YEAR: 40}},
        "child": {"age": {YEAR: child_age}},
    }
    
    # Get all person IDs
    all_members = ["adult", "child"]
    
    situation = {
        "people": people,
        "families": {
            "family": {"members": all_members}
        },
        "marital_units": {
            "marital_unit": {"members": ["adult"]}
        },
        "tax_units": {
            "tax_unit": {"members": all_members}
        },
        "households": {
            "household": {
                "members": all_members,
                "state_name": {YEAR: "VA"},
            }
        },
        "axes": [[
            {
                "name": "employment_income",
                "min": 0,
                "max": MAX_INCOME,
                "count": 361,
                "period": YEAR,
            }
        ]]
    }
    return situation

In [16]:
def calculate_credits(situation, reform=None):
    baseline_simulation = Simulation(situation=situation)
    reformed_simulation = Simulation(situation=situation, reform=reform)
    baseline_income = baseline_simulation.calculate("household_net_income", YEAR)
    reformed_income = reformed_simulation.calculate("household_net_income", YEAR)
    return baseline_income, reformed_income

In [17]:
def create_fisc_comparison_graph():
    colors = {
        "age_0": "#4d94ff",   # Light blue
        "age_3": "#0066cc",   # Medium blue
        "age_10": "#003366",  # Dark blue
    }

    x = np.linspace(0, MAX_INCOME, 361)
    fig = go.Figure()

    # Calculate results for each child age
    child_ages = [0, 3, 10]
    age_descriptions = {
        0: "Newborn child (age 0)",
        3: "Preschool child (age 3)",
        10: "School-age child (age 10)",
    }

    for child_age in child_ages:
        baseline_income, reformed_income = calculate_credits(
            create_situation(child_age), 
            reform=reform
        )
        # Calculate the difference between reformed and baseline
        income_difference = reformed_income - baseline_income
        
        fig.add_trace(
            go.Scatter(
                x=x,
                y=income_difference,
                mode="lines",
                name=f"{age_descriptions[child_age]}",
                line=dict(color=colors[f"age_{child_age}"]),
            )
        )

    fig.update_layout(
        title="Change in Single Parent Household Income Under the FISC ACT",
        xaxis_title="Parent Earnings",
        yaxis_title="Change in Household Income ($)",
        xaxis=dict(tickformat="$,.0f", range=[0, MAX_INCOME]),
        yaxis=dict(tickformat="$,.0f"),
        legend=dict(
            yanchor="top", 
            y=0.99, 
            xanchor="left", 
            x=1.01,
            font=dict(size=10)
        ),
        height=600,
        width=900,
    )
    
    # Add a horizontal line at y=0 to show where there's no change
    fig.add_hline(y=0, line_dash="dash", line_color="gray", opacity=0.5)
    
    return fig

In [18]:
# Create and display the chart
fig = create_fisc_comparison_graph()
fig = format_fig(fig)
fig.show()

In [19]:
def create_credit_comparison_graph():
    colors = {
        "age_0_ctc": "#4d94ff",      # Light blue
        "age_3_ctc": "#0066cc",      # Medium blue
        "age_10_ctc": "#003366",     # Dark blue
        "age_0_fisc": "#ff9999",     # Light red
        "age_3_fisc": "#ff3333",     # Medium red
        "age_10_fisc": "#cc0000",    # Dark red
    }

    x = np.linspace(0, MAX_INCOME, 441)
    fig = go.Figure()

    # Calculate results for each child age
    child_ages = [0, 3, 10]
    age_descriptions = {
        0: "Newborn child (age 0)",
        3: "Preschool child (age 3)",
        10: "School-age child (age 10)",
    }

    for child_age in child_ages:
        situation = create_situation(child_age)
        baseline_sim = Simulation(situation=situation)
        reformed_sim = Simulation(situation=situation, reform=reform)
        
        # Get baseline CTC
        baseline_ctc = baseline_sim.calculate("ctc_value", YEAR)
        
        # Get FISC credit under reform
        fisc_credit = reformed_sim.calculate("family_income_supplement_credit", YEAR)
        
        # Add baseline CTC trace
        fig.add_trace(
            go.Scatter(
                x=x,
                y=baseline_ctc,
                mode="lines",
                name=f"Baseline CTC: {age_descriptions[child_age]}",
                line=dict(color=colors[f"age_{child_age}_ctc"]),
            )
        )
        
        # Add FISC credit trace
        fig.add_trace(
            go.Scatter(
                x=x,
                y=fisc_credit,
                mode="lines",
                name=f"FISC Credit: {age_descriptions[child_age]}",
                line=dict(color=colors[f"age_{child_age}_fisc"]),
            )
        )

    fig.update_layout(
        title="Comparison of Baseline CTC and FISC Credit by Child Age",
        xaxis_title="Parent Earnings",
        yaxis_title="Credit Amount ($)",
        xaxis=dict(tickformat="$,.0f", range=[0, MAX_INCOME]),
        yaxis=dict(tickformat="$,.0f"),
        legend=dict(
            yanchor="top", 
            y=0.99, 
            xanchor="left", 
            x=1.01,
            font=dict(size=10)
        ),
        height=600,
        width=900,
    )
    
    return fig

In [20]:
fig = create_credit_comparison_graph()
fig = format_fig(fig)
fig.show()